# Сравнение запросов для работы с GPT от Yandex и OpenAI

In [4]:
!pip install python-dotenv

Импортируем настройки для работы с yandex и openai

In [7]:
from pydantic_settings import BaseSettings
import logging
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path='../.env')

log_levels = {
    'DEBUG': logging.DEBUG,
    'INFO': logging.INFO,
    'WARNING': logging.WARNING,
    'ERROR': logging.ERROR
}



class Settings(BaseSettings):
    api_name: str = "NLP API"
    logging_level: str = "INFO"

    sentry_dsn: str = ...
    sentry_enable_tracing: bool = True

    elastic_host: str = ...
    elastic_port: int = 9200
    
    project_name: str = ...
    sqlite_path: str = ...
    postgres_version: str = ...
    postgres_ports: str = ...
    postgres_port: str = ...
    postgres_host: str = ...
    postgres_user: str = ...
    postgres_password: str = ...
    postgres_schema_2: str = ...
    postgres_db: str = ...
    postgres_name: str = ...
    elastic_movies_index: str = ...
    stack_version: str = ...
    elastic_cluster_name: str = ...
    memory_lock: str = ...
    elastic_ports: str = ...
    redis_ports: str = ...
    redis_host: str = ...
    redis_port: str = ...
    logstash_ports1: str = ...
    logstash_ports2: str = ...
    logstash_ports3: str = ...
    kibana_ports: str = ...
    es_public_host: str = ...
    logstash_public_host: str = ...
    logstash_host: str = ...
    logstash_port: str = ...
    kibana_public_host: str = ...
    kibana_public_url: str = ...
    elasticsearch_host: str = ...
    es_url: str = ...
    
    yc_api_key: str = ...

    def get_logging_level(self) -> int:
        return log_levels.get(self.logging_level, logging.INFO)

    class Config:
        env_file = '../.env'


settings = Settings()

In [14]:
from langchain import LLMChain, PromptTemplate
from langchain_community.vectorstores import ElasticVectorSearch
from langchain_community.llms import YandexGPT
import time

# YANDEX GPT 3

## Пробуем запросить готовый запрос для Elasticsearch на английском языке

In [15]:
template = """
You are an assistant that converts natural language queries into Elasticsearch queries.
Convert the following natural language query to an Elasticsearch query:

Query: {query}

Please translate any lanquage to english if needed. Write please only the Elasticsearch query.

There are 3 indices: movies, genres and persons. Persons could be directors or actors.
"""

prompt = PromptTemplate(template=template, input_variables=["query"])
llm = YandexGPT(api_key=settings.yc_api_key,
                model_uri="gpt://b1gsfbseql2uigg6q996/yandexgpt/latest")
llm_chain = prompt | llm
current_time = time.time()
query = "Найти фильмы с участием актера Джонни Деппа"

elasticsearch_query = llm_chain.invoke(query)
print (f'Время выполнения: {time.time() - current_time} секунд')
print('='*10)
print(elasticsearch_query)


Время выполнения: 12.824540853500366 секунд
`{"query": {"match": {"persons.name": "Джонни Депп"}}}`


Время выполнения более 4,5 секунд. Нас не устраивает такой результат.

Также не переводятся запросы

## Пробуем запросить готовый запрос для Elasticsearch на руссом языке

In [16]:
template = """
Вы помощник, который преобразует запросы на естественном языке в запросы Elasticsearch.
Преобразуйте следующий запрос на естественном языке в запрос Elasticsearch:

Запрос: {query}

Пожалуйста, переведите любой язык на английский, если это необходимо. Напишите, пожалуйста, только запрос Elasticsearch.

Существует 3 индекса: movies (фильмы), genres (жанры) и persons (люди). Люди могут быть режиссерами или актерами.
"""

prompt = PromptTemplate(template=template, input_variables=["query"])
llm = YandexGPT(api_key=settings.yc_api_key,
                model_uri="gpt://b1gsfbseql2uigg6q996/yandexgpt/latest")
llm_chain = prompt | llm
current_time = time.time()
query = "Найти фильмы с участием актера Джонни Деппа"

elasticsearch_query = llm_chain.invoke(query)
print (f'Время выполнения: {time.time() - current_time} секунд')
print('='*10)
print(elasticsearch_query)


Время выполнения: 9.2378408908844 секунд
```
{
  "query": {
    "term": {
      "persons.name": "Johnny Depp"
    }
  },
  "size": 10,
  "from": 0,
  "_source": "title",
  "sort": [
    {
      "_score": {
        "order": "desc"
      }
    }
  ],
  "aggs": {
    "genres": {
      "terms": {
        "field": "genres.id"
      },
      "size": 5
    }
  }
}
```


Время выполнения запроса на руссом языке более 4,5 секунд. Нас не устраивает такой результат.
Хотя воспиятие запроса лучше

In [17]:
## Вынесем объявление LLM в отдельную ячейку

In [77]:
llm = YandexGPT(api_key=settings.yc_api_key,
                model_uri="gpt://b1gsfbseql2uigg6q996/yandexgpt/latest",
                top_p=0.1,
                temperature=0.1,
                max_tokens=20)

In [34]:
template = """
Определи с поиском чего связан следущий запрос: поиск фильмов, поиск жанров или поиск персон.
Запрос: {query}
Если ты сомневаешься — отвечай none.
У тебя есть 4 варианта ответа: movies, genres, persons, none. Ответь, пожалуйста, одним словом.
"""
prompt = PromptTemplate(template=template, input_variables=["query"])
llm_chain = prompt | llm
current_time = time.time()
queries = ["Найти фильмы с участием актера Джонни Деппа",
           "Найти фильмы без титров",
           "Найти улиточек с именем Наталии Деппа",
           "Что-нибудь про войну"]

for query in queries:
    current_time = time.time()
    search_type = llm_chain.invoke(query)
    print(f'Время выполнения: {time.time() - current_time} секунд')
    print(search_type)
    print('=' * 10)




Время выполнения: 1.5709388256072998 секунд
movies
Время выполнения: 1.1903228759765625 секунд
movies
Время выполнения: 2.8684239387512207 секунд
persons
Время выполнения: 2.262669801712036 секунд
movies


Скорость ответа уже преемлемая


In [36]:
template = """
Определи содержится ли в данном запросе уточнение по свойству: имя человека
Запрос: {query}
Если ты сомневаешься — отвечай none.
Если есть — ответь через запятую строками для фильтра переведенными на английский язык.
Без лишних слов, только значения фильтра через запятую или none
"""
prompt = PromptTemplate(template=template, input_variables=["query"])
llm_chain = prompt | llm
current_time = time.time()
queries = ["Найти фильмы с участием актера Джонни Деппа",
           "Найти фильмы без титров",
           "Найти улиточек с именем Наталии Деппа",
           "Что-нибудь про войну"]

for query in queries:
    current_time = time.time()
    search_type = llm_chain.invoke(query)
    print(f'Время выполнения: {time.time() - current_time} секунд')
    print(search_type)
    print('=' * 10)




Время выполнения: 2.1643247604370117 секунд
Johnny Depp
Время выполнения: 2.9543960094451904 секунд
none
Время выполнения: 3.769972801208496 секунд
«snail, name = «Natalia Depp».
Время выполнения: 1.0787639617919922 секунд
none


In [40]:
template = """
Определи содержится ли в данном запросе уточнение по упорядочиванию результатов по рейтингу
Запрос: {query}
Если ты сомневаешься — отвечай none.
Отвечай отднословно: asc, desc или none
"""
prompt = PromptTemplate(template=template, input_variables=["query"])
llm_chain = prompt | llm
current_time = time.time()
queries = ["Найти фильмы с участием актера Джонни Деппа",
           "Найти фильмы без титров",
           "Найти улиточек с именем Наталии Деппа",
           "Что-нибудь про войну c высоким рейтингом"]

for query in queries:
    current_time = time.time()
    search_type = llm_chain.invoke(query)
    print(f'Время выполнения: {time.time() - current_time} секунд')
    print(search_type)
    print('=' * 10)




Время выполнения: 2.474492311477661 секунд
none
Время выполнения: 1.1037812232971191 секунд
none
Время выполнения: 1.4105260372161865 секунд
none
Время выполнения: 1.1551711559295654 секунд
desc


In [41]:
template = """
Определи содержится ли в данном запросе уточнение по дате выхода фильма
Запрос: {query}
Если ты сомневаешься — отвечай none.
Отвечай в формате: gte:2020-01-01, lte:2021-01-01 или none
"""
prompt = PromptTemplate(template=template, input_variables=["query"])
llm_chain = prompt | llm
current_time = time.time()
queries = ["Найти фильмы с участием актера Джонни Деппа",
           "Найти фильмы без титров",
           "Найти улиточек с именем Наталии Деппа после 2020 года",
           "Что-нибудь про войну c высоким рейтингом"]

for query in queries:
    current_time = time.time()
    search_type = llm_chain.invoke(query)
    print(f'Время выполнения: {time.time() - current_time} секунд')
    print(search_type)
    print('=' * 10)




Время выполнения: 1.1772480010986328 секунд
none
Время выполнения: 5.230074882507324 секунд
none
Время выполнения: 2.9710099697113037 секунд
gte:2020-01-01
Время выполнения: 1.7493548393249512 секунд
none


In [42]:
template = """
Определи содержится ли в данном запросе уточнение по жанру или жанрам фильма
Запрос: {query}
Если ты сомневаешься — отвечай none.
допустимые значения: action, adventure, animation, biography, comedy, crime, drama, family, fantasy, history, horror, music, musical, mystery, romance, sci-fi, sport, thriller, war, western
Отвечай в формате: значений через запятую или none
"""
prompt = PromptTemplate(template=template, input_variables=["query"])
llm_chain = prompt | llm
current_time = time.time()
queries = ["Найти фильмы с участием актера Джонни Деппа",
           "Найти фильмы без титров",
           "Найти улиточек с именем Наталии Деппа после 2020 года",
           "Что-нибудь про войну c высоким рейтингом"]

for query in queries:
    current_time = time.time()
    search_type = llm_chain.invoke(query)
    print(f'Время выполнения: {time.time() - current_time} секунд')
    print(search_type)
    print('=' * 10)




Время выполнения: 1.5728259086608887 секунд
none
Время выполнения: 7.150744199752808 секунд
none
Время выполнения: 6.534531116485596 секунд
none
Время выполнения: 1.2712318897247314 секунд
war


In [80]:
template = """
Определи содержатся ли в запросе слова, которые могут быть частью названия фильма

Если такие есть — перечисли их через запятую на английском. Если нет или сомневаешься — none
Отвечай в формате слова через запятую или none. Без окружающего текста и пояснений.

Запрос: {query}
"""
prompt = PromptTemplate(template=template, input_variables=["query"])
llm_chain = prompt | llm
current_time = time.time()
queries = ["Найти фильмы про звездные войны",
           "Фильм спиздили",
           "Найти мультфильмы про звездные войны",
           "Что-нибудь про войну c высоким рейтингом"]

for query in queries:
    current_time = time.time()
    search_type = llm_chain.invoke(query)
    print(f'Время выполнения: {time.time() - current_time} секунд')
    print(search_type)
    print('=' * 10)




Время выполнения: 1.811396837234497 секунд
Star, Wars
Время выполнения: 6.643424034118652 секунд
none
Время выполнения: 1.3858659267425537 секунд
Star, Wars
Время выполнения: 5.940780162811279 секунд
none


In [82]:
template = """
Определи содержатся ли в запросе слова, которые могут быть частью описания фильма

Если такие есть — перечисли их через запятую на английском. Если нет или сомневаешься — none
Отвечай в формате слова через запятую или none. Без окружающего текста и пояснений.

Запрос: {query}
"""
prompt = PromptTemplate(template=template, input_variables=["query"])
llm_chain = prompt | llm
current_time = time.time()
queries = ["Найти фильмы про звездные войны",
           "Фильм спиздили",
           "Найти улиточек с именем Наталии Деппа после 2020 года",
           "Что-нибудь про войну c высоким рейтингом"]

for query in queries:
    current_time = time.time()
    search_type = llm_chain.invoke(query)
    print(f'Время выполнения: {time.time() - current_time} секунд')
    print(search_type)
    print('=' * 10)




Время выполнения: 10.689929008483887 секунд
Star, Wars
Время выполнения: 2.6427369117736816 секунд
К сожалению, я не могу ничего сказать об этом. Давайте сменим тему?
Время выполнения: 1.220548152923584 секунд
none
Время выполнения: 3.5765650272369385 секунд
war


# YANDEX GPT 2

In [83]:
llm = YandexGPT(api_key=settings.yc_api_key,
                model_uri="gpt://b1gsfbseql2uigg6q996/yandexgpt-lite/latest",
                top_p=0.1,
                temperature=0.1,
                max_tokens=20)

In [44]:
template = """
Определи с поиском чего связан следущий запрос: поиск фильмов, поиск жанров или поиск персон.
Запрос: {query}
Если ты сомневаешься — отвечай none.
У тебя есть 4 варианта ответа: movies, genres, persons, none. Ответь, пожалуйста, одним словом.
"""
prompt = PromptTemplate(template=template, input_variables=["query"])
llm_chain = prompt | llm
current_time = time.time()
queries = ["Найти фильмы с участием актера Джонни Деппа",
           "Найти фильмы без титров",
           "Найти улиточек с именем Наталии Деппа",
           "Что-нибудь про войну"]

for query in queries:
    current_time = time.time()
    search_type = llm_chain.invoke(query)
    print(f'Время выполнения: {time.time() - current_time} секунд')
    print(search_type)
    print('=' * 10)




Время выполнения: 1.2113327980041504 секунд
movies
Время выполнения: 1.196167230606079 секунд
movies
Время выполнения: 1.3221569061279297 секунд
none
Время выполнения: 1.2123713493347168 секунд
movies


In [45]:
template = """
Определи содержится ли в данном запросе уточнение по свойству: имя человека
Запрос: {query}
Если ты сомневаешься — отвечай none.
Если есть — ответь через запятую строками для фильтра переведенными на английский язык.
Без лишних слов, только значения фильтра через запятую или none
"""
prompt = PromptTemplate(template=template, input_variables=["query"])
llm_chain = prompt | llm
current_time = time.time()
queries = ["Найти фильмы с участием актера Джонни Деппа",
           "Найти фильмы без титров",
           "Найти улиточек с именем Наталии Деппа",
           "Что-нибудь про войну"]

for query in queries:
    current_time = time.time()
    search_type = llm_chain.invoke(query)
    print(f'Время выполнения: {time.time() - current_time} секунд')
    print(search_type)
    print('=' * 10)




Время выполнения: 1.5996289253234863 секунд
none
Время выполнения: 1.2463340759277344 секунд
В данном запросе нет уточнения по свойству «имя человека».

Если вы имели в виду
Время выполнения: 1.176764965057373 секунд
В данном запросе нет уточнения по свойству «имя человека».

Если вы имели в виду
Время выполнения: 1.4015171527862549 секунд
none

Если вы имели в виду что-то другое, уточните запрос, пожалуйста.


In [46]:
template = """
Определи содержится ли в данном запросе уточнение по упорядочиванию результатов по рейтингу
Запрос: {query}
Если ты сомневаешься — отвечай none.
Отвечай отднословно: asc, desc или none
"""
prompt = PromptTemplate(template=template, input_variables=["query"])
llm_chain = prompt | llm
current_time = time.time()
queries = ["Найти фильмы с участием актера Джонни Деппа",
           "Найти фильмы без титров",
           "Найти улиточек с именем Наталии Деппа",
           "Что-нибудь про войну c высоким рейтингом"]

for query in queries:
    current_time = time.time()
    search_type = llm_chain.invoke(query)
    print(f'Время выполнения: {time.time() - current_time} секунд')
    print(search_type)
    print('=' * 10)




Время выполнения: 3.6064298152923584 секунд
none
Время выполнения: 1.1230320930480957 секунд
Если запрос звучит так, как вы его привели, и нет дополнительных уточнений, то ответ **none
Время выполнения: 0.974682092666626 секунд
none
Время выполнения: 1.0474810600280762 секунд
Если предположить, что по контексту запроса речь идёт о запросе к поисковой системе, то уточнение по


In [47]:
template = """
Определи содержится ли в данном запросе уточнение по дате выхода фильма
Запрос: {query}
Если ты сомневаешься — отвечай none.
Отвечай в формате: gte:2020-01-01, lte:2021-01-01 или none
"""
prompt = PromptTemplate(template=template, input_variables=["query"])
llm_chain = prompt | llm
current_time = time.time()
queries = ["Найти фильмы с участием актера Джонни Деппа",
           "Найти фильмы без титров",
           "Найти улиточек с именем Наталии Деппа после 2020 года",
           "Что-нибудь про войну c высоким рейтингом"]

for query in queries:
    current_time = time.time()
    search_type = llm_chain.invoke(query)
    print(f'Время выполнения: {time.time() - current_time} секунд')
    print(search_type)
    print('=' * 10)




Время выполнения: 1.0497169494628906 секунд
none
Время выполнения: 1.3605351448059082 секунд
none
Время выполнения: 1.0176358222961426 секунд
none
Время выполнения: 1.1078910827636719 секунд
none


In [48]:
template = """
Определи содержится ли в данном запросе уточнение по жанру или жанрам фильма
Запрос: {query}
Если ты сомневаешься — отвечай none.
допустимые значения: action, adventure, animation, biography, comedy, crime, drama, family, fantasy, history, horror, music, musical, mystery, romance, sci-fi, sport, thriller, war, western
Отвечай в формате: значений через запятую или none
"""
prompt = PromptTemplate(template=template, input_variables=["query"])
llm_chain = prompt | llm
current_time = time.time()
queries = ["Найти фильмы с участием актера Джонни Деппа",
           "Найти фильмы без титров",
           "Найти улиточек с именем Наталии Деппа после 2020 года",
           "Что-нибудь про войну c высоким рейтингом"]

for query in queries:
    current_time = time.time()
    search_type = llm_chain.invoke(query)
    print(f'Время выполнения: {time.time() - current_time} секунд')
    print(search_type)
    print('=' * 10)




Время выполнения: 1.1000089645385742 секунд
none
Время выполнения: 1.4413621425628662 секунд
none
Время выполнения: 10.614950180053711 секунд
none
Время выполнения: 1.210057020187378 секунд
action, war


In [84]:
template = """
Определи содержатся ли в запросе слова, которые могут быть частью названия фильма

Если такие есть — перечисли их через запятую на английском. Если нет или сомневаешься — none
Отвечай в формате слова через запятую или none. Без окружающего текста и пояснений.

Запрос: {query}
"""
prompt = PromptTemplate(template=template, input_variables=["query"])
llm_chain = prompt | llm
current_time = time.time()
queries = ["Найти фильмы про звездные войны",
           "Фильм спиздили",
           "Найти мультфильмы про звездные войны",
           "Что-нибудь про войну c высоким рейтингом"]

for query in queries:
    current_time = time.time()
    search_type = llm_chain.invoke(query)
    print(f'Время выполнения: {time.time() - current_time} секунд')
    print(search_type)
    print('=' * 10)




Время выполнения: 3.706592082977295 секунд
none
Время выполнения: 1.270448923110962 секунд
none
Время выполнения: 1.3024439811706543 секунд
none
Время выполнения: 1.1147210597991943 секунд
**none**


In [85]:
template = """
Определи содержатся ли в запросе слова, которые могут быть частью описания фильма

Если такие есть — перечисли их через запятую на английском. Если нет или сомневаешься — none
Отвечай в формате слова через запятую или none. Без окружающего текста и пояснений.

Запрос: {query}
"""
prompt = PromptTemplate(template=template, input_variables=["query"])
llm_chain = prompt | llm
current_time = time.time()
queries = ["Найти фильмы про звездные войны",
           "Фильм спиздили",
           "Найти улиточек с именем Наталии Деппа после 2020 года",
           "Что-нибудь про войну c высоким рейтингом"]

for query in queries:
    current_time = time.time()
    search_type = llm_chain.invoke(query)
    print(f'Время выполнения: {time.time() - current_time} секунд')
    print(search_type)
    print('=' * 10)




Время выполнения: 0.8195781707763672 секунд
none
Время выполнения: 0.8576440811157227 секунд
none
Время выполнения: 1.3117640018463135 секунд
**none**
Время выполнения: 1.171189546585083 секунд
combat, drama, historical, war
